In [ ]:
!az login -o table

In [ ]:
!az provider register -n Microsoft.BatchAI
!az provider register -n Microsoft.Batch

In [2]:
!az account list -o table

Name                                 CloudName    SubscriptionId                        State    IsDefault
-----------------------------------  -----------  ------------------------------------  -------  -----------
Marketing Automation                 AzureCloud   03909a66-bef8-4d52-8e9a-a346604e0902  Enabled
Boston Engineering                   AzureCloud   bc4170f0-cc6e-49d2-ba65-bc00a7a4df6b  Enabled
ADLTrainingMS                        AzureCloud   ace74b35-b0de-428b-a1d9-55459d7a6e30  Enabled
Ads Eng Big Data Subscription        AzureCloud   d4231148-f093-4473-85de-8c4d19247495  Enabled
Data Wrangling Preview               AzureCloud   215613ac-9dfb-488c-be46-c387e999b127  Enabled
Data Wrangling development           AzureCloud   35f16a99-532a-4a47-9e93-00305f6c40f2  Enabled
AzureML Client PROD                  AzureCloud   4a0f8d7f-c197-476b-95e1-fafcd77f12e4  Enabled
R portal - Production                AzureCloud   2569dcd9-a995-4b78-ae22-58a1015fd0f7  Enabled
Project Vienna D

In [ ]:
!az account set -s "<YOURTEAM>"

In [3]:
!az --version

azure-cli (2.0.32)

acr (2.0.24)
acs (2.0.32)
advisor (0.5.1)
ams (0.1.0)
appservice (0.1.32)
backup (1.1.1)
batch (3.2.2)
batchai (0.2.2)
billing (0.1.8)
cdn (0.0.14)
cloud (2.0.13)
cognitiveservices (0.1.13)
command-modules-nspkg (2.0.1)
configure (2.0.15)
consumption (0.3.1)
container (0.1.23)
core (2.0.32)
cosmosdb (0.1.21)
dla (0.0.19)
dls (0.0.21)
dms (0.0.1)
eventgrid (0.1.12)
eventhubs (0.1.3)
extension (0.0.13)
feedback (2.1.1)
find (0.2.9)
interactive (0.3.20)
iot (0.1.20)
keyvault (2.0.21)
lab (0.0.22)
monitor (0.1.6)
network (2.1.0)
nspkg (3.0.2)
profile (2.0.24)
rdbms (0.2.2)
redis (0.2.13)
reservations (0.1.2)
resource (2.0.28)
role (2.0.23)
servicebus (0.1.2)
servicefabric (0.0.12)
sql (2.0.25)
storage (2.0.32)
vm (2.0.31)

Python location '/opt/az/bin/python3'
Extensions directory '/home/hoaphumanoid/.azure/cliextensions'

Python (Linux) 3.6.1 (default, May  1 2018, 17:46:32) 
[GCC 4.7.2]

Legal docs and information: aka.ms/AzureCliLegal




In [4]:
import os
import uuid
import random
import json

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

LOCATION = 'eastus' # We are setting everything up in East US
                    # Be aware that you need to set things up in a region that has GPU VMs (N-Series)

short_uuid = str(uuid.uuid4())[:8]
GROUP_NAME = "batch{uuid}rg".format(uuid=short_uuid)
STORAGE_ACCOUNT_NAME = "batch{uuid}st".format(uuid=short_uuid)
CONTAINER_NAME = "batch{uuid}container".format(uuid=short_uuid)
FILESHARE_NAME = "batch{uuid}share".format(uuid=short_uuid)
NFS_NAME="batch{uuid}nfs".format(uuid=short_uuid)

In [5]:
!az group create -n $GROUP_NAME -o table -l $LOCATION

Location    Name
----------  ---------------
eastus      batch80ca5fa5rg


In [6]:
json_data = !az storage account create -l $LOCATION -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME --sku Standard_LRS
print('Storage account {} provisioning state: {}'.format(STORAGE_ACCOUNT_NAME, json.loads(''.join(json_data))['provisioningState']))

Storage account batch80ca5fa5st provisioning state: Succeeded


In [7]:
json_data = !az storage account keys list -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME
storage_account_key = json.loads(''.join(json_data))[0]['value']

In [8]:
!az storage share create --account-name $STORAGE_ACCOUNT_NAME \
--account-key $storage_account_key --name $FILESHARE_NAME

{
  "created": true
}


In [9]:
!az storage directory create --share-name $FILESHARE_NAME  --name scripts \
--account-name $STORAGE_ACCOUNT_NAME --account-key $storage_account_key

{
  "created": true
}


In [10]:
!az configure --defaults location=$LOCATION

In [11]:
!az configure --defaults group=$GROUP_NAME

In [12]:
%env AZURE_STORAGE_ACCOUNT $STORAGE_ACCOUNT_NAME

env: AZURE_STORAGE_ACCOUNT=batch80ca5fa5st


In [13]:
%env AZURE_STORAGE_KEY=$storage_account_key

env: AZURE_STORAGE_KEY=0b5FROnd9S+VNiQDDMl9/tlPkCje003NE14/ks9n9DUBf+x7TbEC2dq2wwUOGq+jDyEhclN3/WqPpbpWmD4IaQ==


In [17]:
!az batchai file-server create -n $NFS_NAME --disk-count 4 --disk-size 250 -s Standard_DS4_v2 -u mat -p d13NHAL! -g $GROUP_NAME -l $LOCATION --storage-sku Premium_LRS 


{- Finished ..
  "additionalProperties": {},
  "creationTime": "2018-06-04T16:03:03.580000+00:00",
  "dataDisks": {
    "additionalProperties": {},
    "cachingType": "none",
    "diskCount": 4,
    "diskSizeInGb": 250,
    "storageAccountType": "Premium_LRS"
  },
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/batch80ca5fa5rg/providers/Microsoft.BatchAI/fileservers/batch80ca5fa5nfs",
  "location": "eastus",
  "mountSettings": {
    "additionalProperties": {},
    "fileServerInternalIp": "10.0.0.4",
    "fileServerPublicIp": "40.117.128.168",
    "fileServerType": "nfs",
    "mountPoint": "/data"
  },
  "name": "batch80ca5fa5nfs",
  "provisioningState": "succeeded",
  "provisioningStateTransitionTime": "2018-06-04T16:06:51.732000+00:00",
  "resourceGroup": "batch80ca5fa5rg",
  "sshConfiguration": {
    "additionalProperties": {},
    "publicIpsToAllow": null,
    "userAccountSettings": {
      "additionalProperties": {},
      "adminUserName": "mat",
      "

In [18]:
!az batchai file-server list -o table

Name              Resource Group    Size             Disks       Public IP       Internal IP    Type    Mount Point
----------------  ----------------  ---------------  ----------  --------------  -------------  ------  -------------
batch80ca5fa5nfs  batch80ca5fa5rg   Standard_DS4_v2  4 x 250 Gb  40.117.128.168  10.0.0.4       nfs     /data


In [19]:
%%writefile nodeprep.sh
#!/usr/bin/env bash
wget https://gist.githubusercontent.com/msalvaris/073c28a9993d58498957294d20d74202/raw/916eefe763c71da49d8ed41cb8474bdc8021af33/install_azcopy
chmod 777 install_azcopy
sudo ./install_azcopy

mkdir -p /data/imagenet

azcopy --source https://datasharesa.blob.core.windows.net/imagenet/cntk/val_map.txt \
    --destination  /data/imagenet/val_map.txt \
    --source-sas '?sp=r&st=2018-05-21T13:52:05Z&se=2025-05-21T21:52:05Z&spr=https,http&sv=2017-07-29&sig=YuPsiiPJ%2FUQTvs8U9i1uYuna8bQW5SiIxDGvl27wbyk%3D&sr=b'    \
    --quiet
       
azcopy --source https://datasharesa.blob.core.windows.net/imagenet/validation.tar.gz \
        --destination  /data/imagenet/validation.tar.gz\
        --source-sas "?se=2025-01-01&sp=r&sv=2017-04-17&sr=b&sig=zy8L4shZa3XXBe152hPnhXsyfBqCufDOz01a9ZHWU28%3D"\
        --quiet

azcopy --source https://datasharesa.blob.core.windows.net/imagenet/cntk/train_map.txt \
        --destination  /data/imagenet/train_map.txt \
        --source-sas '?sp=r&st=2018-05-21T13:59:25Z&se=2025-05-21T21:59:25Z&spr=https&sv=2017-07-29&sig=Nl5%2FquqbqkaeJeZnM%2BNeX2T9mmMf%2BoLAJtV1OXugkdI%3D&sr=b' \
        --quiet        
        
        
azcopy --source https://datasharesa.blob.core.windows.net/imagenet/train.tar.gz \
        --destination  /data/imagenet/train.tar.gz\
        --source-sas "?se=2025-01-01&sp=r&sv=2017-04-17&sr=b&sig=qP%2B7lQuFKHo5UhQKpHcKt6p5fHT21lPaLz1O/vv4FNU%3D"\
        --quiet

azcopy --source https://datasharesa.blob.core.windows.net/imagenet/cntk/ImageNet1K_mean.xml \
        --destination /data/imagenet/ImageNet1K_mean.xml \
        --source-sas '?sp=r&st=2018-05-21T14:00:22Z&se=2025-05-21T22:00:22Z&spr=https&sv=2017-07-29&sig=iZgfdPr1vgEHqLJPdOteBulaUQudh6etVH%2FzJ9wGGPU%3D&sr=b' \
        --quiet
        
cd /data/imagenet
tar -xvzf train.tar.gz
tar -xzvf validation.tar.gz

Overwriting nodeprep.sh


Copy the file over and run it on the NFS VM. This will install azcopy and download and prepare the data

```bash
scp nodeprep.sh mat@<VM_IP>:~/
ssh mat@<VM_IP> sudo chmod 777 ~/nodeprep.sh
ssh mat@<VM_IP> ./nodeprep.sh
```


In [161]:
!az storage file upload --share-name $FILESHARE_NAME --source ../execution/ImagenetEstimatorCNTK.py --path scripts

Finished[#############################################################]  100.0000%


In [21]:
!az storage file upload --share-name $FILESHARE_NAME --source ../execution/resnet_models.py --path scripts

Finished[#############################################################]  100.0000%


In [123]:
!az batchai cluster create \
--name nc24r \
--image UbuntuLTS \
--vm-size Standard_NC24rs_v3 \
--min 8 --max 8 \
--afs-name $FILESHARE_NAME \
--afs-mount-path extfs \
--user-name mat \
--password dnstvxrz \
--storage-account-name $STORAGE_ACCOUNT_NAME \
--storage-account-key $storage_account_key \
--nfs $NFS_NAME \
--nfs-mount-path nfs

{- Finished ..
  "additionalProperties": {},
  "allocationState": "resizing",
  "allocationStateTransitionTime": "2018-06-05T20:39:27.398000+00:00",
  "creationTime": "2018-06-05T20:39:27.398000+00:00",
  "currentNodeCount": 0,
  "errors": null,
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/batch80ca5fa5rg/providers/Microsoft.BatchAI/clusters/nc24r",
  "location": "eastus",
  "name": "nc24r",
  "nodeSetup": {
    "additionalProperties": {},
    "mountVolumes": {
      "additionalProperties": {},
      "azureBlobFileSystems": null,
      "azureFileShares": [
        {
          "accountName": "batch80ca5fa5st",
          "additionalProperties": {},
          "azureFileUrl": "https://batch80ca5fa5st.file.core.windows.net/batch80ca5fa5share",
          "credentials": {
            "accountKey": null,
            "accountKeySecretReference": null,
            "additionalProperties": {}
          },
          "directoryMode": "0777",
          "fileMode": "0777

In [44]:
!az batchai cluster show -n nc24r

{
  "additionalProperties": {},
  "allocationState": "resizing",
  "allocationStateTransitionTime": "2018-06-04T20:22:21.168000+00:00",
  "creationTime": "2018-06-04T20:22:21.168000+00:00",
  "currentNodeCount": 0,
  "errors": null,
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/batch80ca5fa5rg/providers/Microsoft.BatchAI/clusters/nc24r",
  "location": "eastus",
  "name": "nc24r",
  "nodeSetup": {
    "additionalProperties": {},
    "mountVolumes": {
      "additionalProperties": {},
      "azureBlobFileSystems": null,
      "azureFileShares": [
        {
          "accountName": "batch80ca5fa5st",
          "additionalProperties": {},
          "azureFileUrl": "https://batch80ca5fa5st.file.core.windows.net/batch80ca5fa5share",
          "credentials": {
            "accountKey": null,
            "accountKeySecretReference": null,
            "additionalProperties": {}
          },
          "directoryMode": "0777",
          "fileMode": "0777",
          

In [167]:
!az batchai cluster list -o table

In [104]:
!az batchai cluster list-nodes -n nc24r -o table

ID                                 IP               Port
---------------------------------  -------------  ------
tvm-4155946844_1-20180605t151846z  13.68.131.112   50006
tvm-4155946844_2-20180605t151846z  13.68.131.112   50002
tvm-4155946844_3-20180605t151846z  13.68.131.112   50005
tvm-4155946844_4-20180605t151846z  13.68.131.112   50001
tvm-4155946844_5-20180605t151846z  13.68.131.112   50004
tvm-4155946844_6-20180605t151846z  13.68.131.112   50000
tvm-4155946844_7-20180605t151846z  13.68.131.112   50007
tvm-4155946844_8-20180605t151846z  13.68.131.112   50003


In [162]:
jobs_dict = {
  "$schema": "https://raw.githubusercontent.com/Azure/BatchAI/master/schemas/2017-09-01-preview/job.json",
  "properties": {
    "nodeCount": 8,
    "cntkSettings": {
            "pythonScriptFilePath": "$AZ_BATCHAI_INPUT_SCRIPTS/ImagenetEstimatorCNTK.py",
            "processCount": 32
        },
    "stdOutErrPathPrefix": "$AZ_BATCHAI_MOUNT_ROOT/extfs",
    "inputDirectories": [{
        "id": "SCRIPTS",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/extfs/scripts"
      },
      {
        "id": "TRAIN",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/imagenet",
      },
      {
        "id": "TEST",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/imagenet",
      },
    ],
    "outputDirectories": [{
        "id": "MODEL",
        "pathPrefix": "$AZ_BATCHAI_MOUNT_ROOT/extfs",
        "pathSuffix": "Models"
    }],
    "containerSettings": {
      "imageSourceRegistry": {
        "image": "hoaphumanoid/cntk:2.5.1-cuda9.0-cudnn7.0.5-libopenmpi-py3.5-pip"
      }
    }
  }
}

In [163]:
write_json_to_file(jobs_dict, 'job.json')

In [164]:
!az batchai job create -n cntk_pip10 --cluster-name nc24r -c job.json -g $GROUP_NAME

{- Finished ..
  "additionalProperties": {},
  "caffeSettings": null,
  "chainerSettings": null,
  "cluster": {
    "additionalProperties": {},
    "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/batch80ca5fa5rg/providers/Microsoft.BatchAI/clusters/nc24r",
    "resourceGroup": "batch80ca5fa5rg"
  },
  "cntkSettings": {
    "additionalProperties": {},
    "commandLineArgs": null,
    "configFilePath": null,
    "languageType": "Python",
    "processCount": 32,
    "pythonInterpreterPath": null,
    "pythonScriptFilePath": "$AZ_BATCHAI_INPUT_SCRIPTS/ImagenetEstimatorCNTK.py"
  },
  "constraints": {
    "additionalProperties": {},
    "maxWallClockTime": "7 days, 0:00:00"
  },
  "containerSettings": {
    "additionalProperties": {},
    "imageSourceRegistry": {
      "additionalProperties": {},
      "credentials": null,
      "image": "hoaphumanoid/cntk:2.5.1-cuda9.0-cudnn7.0.5-libopenmpi-py3.5-pip",
      "serverUrl": null
    }
  },
  "creationTime": "2018-06-

In [50]:
!az batchai job list -o table -g $GROUP_NAME

Name            Resource Group    Cluster    Cluster RG       Tool      Nodes  State
--------------  ----------------  ---------  ---------------  ------  -------  -------
cntk_benchmark  batch80ca5fa5rg   nc24r      batch80ca5fa5rg  cntk          8  queued


In [ ]:
!az batchai job file stream -n cntk_benchmark -g $GROUP_NAME -f stdout.txt

In [116]:
!az batchai job delete --name cntk_benchmark_pip3 -y

inished ..

In [ ]:
!az configure --defaults group=''
!az configure --defaults location=''

In [166]:
!az batchai cluster delete --name nc24r -g $GROUP_NAME -y

inished ..

In [ ]:
!az group delete --name $GROUP_NAME -y